In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
import sys

# ==============================================================================
# 1. CONSTANTS AND DATA SETUP (Crucial for fixing ValueError)
# ==============================================================================

IMAGE_SIZE = 256  # Must match the size expected by the model architecture
BATCH_SIZE = 32
EPOCHS = 20
DATA_DIR = 'Potato/Train'
INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
NUM_CLASSES = 3  # Assuming 3 classes: Healthy, EarlyBlight, LateBlight

# --- Data Loading (Using your provided method) ---
print(f"Loading data from directory: {DATA_DIR}...")
try:
    # 1. Load Training Dataset (80% of data)
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory=DATA_DIR,
        seed=123,
        shuffle=True,
        image_size=(IMAGE_SIZE, IMAGE_SIZE), # Use 256x256
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        subset='training'
    )

    # 2. Load Validation/Test Dataset (20% of data)
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory=DATA_DIR,
        seed=123,
        shuffle=True,
        image_size=(IMAGE_SIZE, IMAGE_SIZE), # Use 256x256
        batch_size=BATCH_SIZE,
        validation_split=0.2,
        subset='validation'
    )
    # NOTE: Since you only have train/validation split from one directory, 
    # we'll use val_ds for both validation and evaluation for simplicity here.
    test_ds = val_ds 

    class_names = train_ds.class_names
    NUM_CLASSES = len(class_names)
    print(f"Number of Classes: {NUM_CLASSES}")

except Exception as e:
    print(f"\nERROR: Data loading failed. Using mock data. ({e})")
    # --- MOCK DATA FALLBACK ---
    NUM_CLASSES = 3
    X_mock = np.random.rand(100, *INPUT_SHAPE).astype('float32')
    y_mock = np.random.randint(0, NUM_CLASSES, 100)
    
    # Create batched tf.data.Dataset objects for the structure below
    train_ds = tf.data.Dataset.from_tensor_slices((X_mock[:80] * 255.0, y_mock[:80])).batch(BATCH_SIZE)
    val_ds = tf.data.Dataset.from_tensor_slices((X_mock[80:] * 255.0, y_mock[80:])).batch(BATCH_SIZE)
    test_ds = val_ds


# --- Normalization and Performance Setup ---
def normalize_and_convert_labels(image, label):
    # Normalize images from [0, 255] to [0, 1]
    image = tf.cast(image / 255.0, tf.float32)
    return image, label

# Apply normalization and caching
train_ds = train_ds.map(normalize_and_convert_labels).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.map(normalize_and_convert_labels).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.map(normalize_and_convert_labels).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Loading data from directory: Potato/Train...
Found 900 files belonging to 3 classes.
Using 720 files for training.
Found 900 files belonging to 3 classes.
Using 180 files for validation.
Number of Classes: 3


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout

# Assuming INPUT_SHAPE = (256, 256, 3) and NUM_CLASSES = 3 are defined globally
def build_vgg16(input_shape=(256, 256, 3), num_classes=3):
    """VGG16 with a custom classification head."""
    # weights=None ensures random initialization (not transfer learning)
    base_vgg = VGG16(weights=None, include_top=False, input_shape=input_shape)
    
    model = models.Sequential(name="VGG16")
    model.add(base_vgg)
    
    # Custom Classification Head
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

# Instantiate the model
vgg16_model = build_vgg16(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)
print("Model definition completed: VGG16")

# Call .summary() on the Keras model object
vgg16_model.summary()

Model definition completed: VGG16


Model: "VGG16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 8, 8, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   134,221,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │        12,291 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 148,948,803 (568.19 MB)

 Trainable params: 148,948,803 (568.19 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
import tensorflow as tf

# --- 2.1. Model Compilation ---

# We use SparseCategoricalCrossentropy as the normalization step did not 
# convert labels to one-hot encoding.
vgg16_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
    metrics=['accuracy']
)
print("Model compiled.")

# --- 2.2. Model Training ---
print(f"\nStarting VGG16 training for {EPOCHS} epochs...")

history_vgg16 = vgg16_model.fit(
    train_ds,
    validation_data=val_ds, # Using val_ds for validation
    verbose=1,
    epochs=EPOCHS,
)

# --- 2.3. Model Evaluation ---
print("\nEvaluating VGG16 on the test dataset...")
scores_vgg16 = vgg16_model.evaluate(test_ds, verbose=0)

# Print the final results
print("\n==============================================")
print("          VGG16 FINAL TEST RESULTS")
print("==============================================")
print(f"VGG16 Test Loss: {scores_vgg16[0]:.4f}")
print(f"VGG16 Test Accuracy: {scores_vgg16[1] * 100:.2f}%")

# Save the history object for later plotting (optional, but recommended)
# history_dict["VGG16"] = history_vgg16

Model compiled.

Starting VGG16 training for 20 epochs...
Epoch 1/20
 5/23 ━━━━━━━━━━━━━━━━━━━━ 4:30 15s/step - accuracy: 0.3020 - loss: 1.7568

KeyboardInterrupt: 